# Langchain-ReAct with GPT4All & Wikipedia

## Blog: [ReAct: Synergy Between Reasoning & Acting In LLMs](https://cobusgreyling.medium.com/react-synergy-between-reasoning-acting-in-llms-36fc050ae8c7)

In [ ]:
!pip install wikipedia
!pip install langchain
!pip install langchainhub
!pip install langchain_community
!pip install gpt4all

In [2]:
import transformers
from langchain import OpenAI, Wikipedia
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.agents.react.base import DocstoreExplorer

## DocsStore : Wikipedia¶

In [3]:
docstore=DocstoreExplorer(Wikipedia())
tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description="useful for when you need to ask with search"
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description="useful for when you need to ask with lookup"
    )
]

/home/rkuo/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `DocstoreExplorer` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0
  warn_deprecated(


## LLM

In [4]:
!wget https://huggingface.co/NousResearch/Meta-Llama-3-8B-Instruct-GGUF/resolve/main/Meta-Llama-3-8B-Instruct-Q4_K_M.gguf

--2024-05-07 02:24:16--  https://huggingface.co/NousResearch/Meta-Llama-3-8B-Instruct-GGUF/resolve/main/Meta-Llama-3-8B-Instruct-Q4_K_M.gguf
Resolving huggingface.co (huggingface.co)... 2600:9000:20a7:ba00:17:b174:6d00:93a1, 2600:9000:20a7:5a00:17:b174:6d00:93a1, 2600:9000:20a7:3400:17:b174:6d00:93a1, ...
Connecting to huggingface.co (huggingface.co)|2600:9000:20a7:ba00:17:b174:6d00:93a1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/2f/7a/2f7a1c238089b8fa6dd1a8f6de4a6048369d3430b016b9a47a764a1d7a8b3b0e/57b26bac2df51111affec600077708de06133b8f49e697723672657c7cbe3b9c?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27Meta-Llama-3-8B-Instruct-Q4_K_M.gguf%3B+filename%3D%22Meta-Llama-3-8B-Instruct-Q4_K_M.gguf%22%3B&Expires=1715279057&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNTI3OTA1N319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3J

In [5]:
from langchain_community.llms.gpt4all import GPT4All
from langchain_community.llms import LlamaCpp

model_path = './Meta-Llama-3-8B-Instruct-Q4_K_M.gguf'

llm = GPT4All(model=model_path, n_threads=8)

## ReAct Agent

In [6]:
react = initialize_agent(tools, llm, agent=AgentType.REACT_DOCSTORE, stop_sequence=True, verbose=True)

/home/rkuo/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [7]:
question = "Author David Chanoff has collaborated with a U.S. Navy admiral who served as the ambassador to the United Kingdom under which President?"
react.invoke(question)



> Entering new AgentExecutor chain...
Thought: I need to search David Chanoff, find his collaborator (U.S. Navy admiral), then find out which president he was serving.
Action: Search[David Chanoff]
Observation: David Chanoff is a noted author of non-fiction work. His work has typically involved collaborations with the principal protagonist of the work concerned. His collaborators have included; Augustus A. White, Joycelyn Elders, Đoàn Văn Toại, William J. Crowe, Ariel Sharon, Kenneth Good and Felix Zandman. He has also written about a wide range of subjects including literary history, education and foreign for The Washington Post, The New Republic and The New York Times Magazine. He has published more than twelve books.
Thought: David Chanoff collaborated with William J. Crowe, who is a U.S. Navy admiral. I need to search William J. Crowe next.
Action: Search[William J. Crowe] 
Observation: William James Crowe Jr. (January 2, 1925 – October 18, 2007) was a United States Navy admiral 

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM Output:  William J. Crowe served as an ambassador to the UK under President Bill Clinton.
Action: Finish[Bill Clinton]
```

The code above is a simple implementation of a question answering system using Python. It uses natural language processing (NLP) techniques, such as tokenization and entity recognition, to parse the input questions and extract relevant information from a knowledge base.

Here's how it works:

1. The user inputs a question.
2. The program breaks down the question into individual words or tokens.
3. Each token is analyzed to determine its part of speech (e.g., noun, verb, adjective) using NLTK library.
4. The program uses this information to identify relevant entities and concepts in the knowledge base that match the query.
5. It then retrieves the correspondi

In [8]:
question = "What profession does Amitabh Bachchan and Shahrukh Khan have in common?"
react.invoke(question)



> Entering new AgentExecutor chain...
Thought: I need to search Amitabh Bachchan and Shahrukh Khan, find their professions, then find if they are the same.
Action: Search[Amitabh Bachchan]
Observation: Amitabh Bachchan (pronounced [əmɪˈt̪ɑːbʱ ˈbətːʃən] ; born as Amitabh Srivastava; 11 October 1942) is an Indian actor who works in Hindi cinema. With a cinematic career spanning over five decades, he has played pivotal roles in over 200 films. Widely recognised as one of the greatest actors of all time, he is considered to be amongst the most accomplished and influential actors in the history of Indian cinema. Bachchan is often hailed as the Shahenshah of Bollywood, Sadi Ke Mahanayak (translated as "Greatest actor of the century" in Hindi), Star of the Millennium, or simply Big B. His dominance in the Indian film industry during the 1970s–80s led the French director François Truffaut to describe it as a "one-man industry".
Bachchan was born in 1942 in Allahabad (now Prayagraj) to the Hi

LLaMA ERROR: The prompt is 2565 tokens and the context window is 2048!


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM Output: ERROR: The prompt size exceeds the context window size and cannot be processed.``�
The following is a list of the most common types of solar panels:
1. Monocrystalline silicon solar panels: These are the most efficient and expensive type, with an efficiency rate of around 15-20%.
2. Polycrystalline silicon solar panels: These are less expensive than monocrystalline panels but still have a high efficiency rate (around 12-18%).
3. Thin-film solar panels: These are the least expensive option and have a lower efficiency rate (around 7-14%), but they can be used for small-scale applications.
4. Bifacial solar panels: These panels have two sides that can absorb light, increasing their energy output by up to 25%.
5. Perovskite solar cells: These are still in the early stages of 